## Preamble

This file is the starting point of the project. Here, we aim to upload the partially cleaned files to a AWS bucket.

To start, the raw files were downloaded from the [National Library of Medicine](https://lhncbc.nlm.nih.gov/ii/tools/SemRep_SemMedDB_SKR/SemMedDB_download.html). Since these are some pretty large files (up to 16GB compressed) and they're officially available for download through the National Library of Medicine, these files are not uploaded with the code. The folders in which they were stored and referenced have been kept if you would like to re-run the code on your own.

This project was completed using the SEMMED v43, 2023, Regular version of these files.

Now, some manual cleaning was required before starting running any Python Code. As any Data Scientist/Analyst knows, many data processing packages like Pandas and Dask can read data in multiple formats, however, certain characters can trip them up. Of these manual changes (which took quite some time), the most important are:
 - Removing special characters like '\\'. These characters were littered across the files and caused a variety of issues when interpreted.
 - Removed some diacritics and accent marks from certain characters. ISO-8859-1 encoding parses some of these correctly and some like 'Ö' incorrectly . Only the problematic ones were removed. 

Lastly, some important notes:
- No official data encoding for the data was provided. ISO-8859-1 is the best match. This was found through visual inspection and manually testing different encodings.
- The Database data doesn't seem to enforce any particular convention. For example, in some entries, a researcher's name will have the appropriate diacritics or accents added to characters. In other entries referencing the same researcher, their name will not contain any diacritics or accents. These differences should be noted and resolved if entries are used.

## Python info

While running `pip install -r requirements.txt` should suffice, all of the necessary packages can be installed by running `pip install` with the following packages:
 - boto3
 - dask["distributed"]
 - jupyter
 - neo4j
 - numpy
 - pandas
 - progressbar
 - pyarrow

Either way, here are the exact packages and versions installed in my Python Virtual Environment. Note the version of Python running as some versions of packages are only restricted to some versions of Python.

In [6]:
print("Python Version:")
!python --version

print("\nInstalled Packages w/ Versions:")
!pip freeze

Python Version:
Python 3.12.4

Installed Packages w/ Versions:
anyio==4.6.0
appnope==0.1.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==24.2.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.1.0
boto3==1.35.34
botocore==1.35.34
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.3.2
click==8.1.7
cloudpickle==3.0.0
comm==0.2.2
dask==2024.9.1
debugpy==1.8.6
decorator==5.1.1
defusedxml==0.7.1
distributed==2024.9.1
executing==2.1.0
fastjsonschema==2.20.0
fqdn==1.5.1
fsspec==2024.9.0
h11==0.14.0
httpcore==1.0.6
httpx==0.27.2
idna==3.10
ipykernel==6.29.5
ipython==8.28.0
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.4
jmespath==1.0.1
json5==0.9.25
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2023.12.1
jupyter==1.1.1
jupyter-console==6.6.3
jupyter-events==0.10.0
jupyter-lsp==2.2.5
jupyter_client==8.6.3
jupyter_core==5.7.2
jupyter_server==2.14.2
jupyter_server_terminals==0.5.3
jupyterlab==4.2.5
jupy

## Important globals

### Changes to Official Site

The next section details some pretty important discrepancies between the data and documentation available of the National Library of Medicine's site.

After finding these discrepancies, I reported them using the through a general "Contact Us" inquiry. Little did I know, one of the lead developers/maintainers of the site would contact me back. Through our email chain, I described the various oddities and received confirmations of them.

The maintainers agreed to update their documentation with the updated details produced from the inquiry; a highlight of my year for sure.

In [ ]:
import os
from pathlib import Path


root = os.path.join(Path.cwd(), "..", "csv")

# Uncompressed files - Used for inspection.
ENTITY_FILE = os.path.join(root, "semmedVER43_2023_R_ENTITY.csv")
PREDICATION_FILE = os.path.join(root, "semmedVER43_2023_R_PREDICATION.csv")
SENTENCE_FILE = os.path.join(root, "semmedVER43_2023_R_SENTENCE.csv")

# Pre-compressed files
zipped = os.path.join(root, "zipped")
ENTITY_ZIP = os.path.join(zipped, "semmedVER43_2023_R_ENTITY.csv.gz")
PREDICATION_ZIP = os.path.join(zipped, "semmedVER43_2023_R_PREDICATION.csv.gz")
SENTENCE_ZIP = os.path.join(zipped, "semmedVER43_2023_R_SENTENCE.csv.gz")

# Column labels for indexing. Found in schema desc(https://lhncbc.nlm.nih.gov/ii/tools/SemRep_SemMedDB_SKR/dbinfo.html)
PREDICATION_COLUMNS = {
    "PREDICATION_ID": str,  # Auto-generated primary key for each unique predication
    "SENTENCE_ID": str,     # Foreign key to the SENTENCE table
    "PMID": str,            # The PubMed identifier of the citation to which the predication belongs
    "PREDICATE": str,       # The string representation of each predicate (for example TREATS, PROCESS_OF)
    "SUBJECT_CUI": str,     # The CUI of the subject of the predication
    "SUBJECT_NAME": str,    # The preferred name of the subject of the predication
    "SUBJECT_SEMTYPE": str, # The semantic type of the subject of the predication
    "SUBJECT_NOVELTY": str, # The novelty of the subject of the predication
    "OBJECT_CUI": str,      # The CUI of the object of the predication
    "OBJECT_NAME": str,     # The preferred name of the object of the predication
    "OBJECT_SEMTYPE": str,  # The semantic type of the object of the predication
    "OBJECT_NOVELTY": str,  # The novelty of the object of the predication
}


SENTENCE_COLUMNS= {
    "SENTENCE_ID": str,               # Auto-generated primary key for each sentence
    "PMID": str,                      # The PubMed identifier of the citation to which the sentence belongs
    "TYPE": str,                      # 'ti' for the title of the citation, 'ab' for the abstract
    "NUMBER": str,                    # The location of the sentence within the title or abstract
    "SENT_START_INDEX": str,          # The character position within the text of the MEDLINE citation of the first character of the sentence  NEW
    "SENT_END_INDEX": str,            # The character position within the text of the MEDLINE citation of the last character of the sentence  NEW
    "SECTION_HEADER": str,            # Section header name of structured abstract (from Version 3.1)
    "NORMALIZED_SECTION_HEADER": str, # Normalized section header name of structured abstract (from Version 3.1)
    "SENTENCE": str,                  # The actual string or text of the sentence
}

ENTITY_COLUMNS = {
    "ENTITY_ID": str,    # Auto-generated primary key for each unique entity
    "SENTENCE_ID": str,  # The foreign key to SENTENCE table
    "CUI": str,          # The CUI of the entity
    "NAME": str,         # The preferred name of the entity
    "TYPE": str,         # The semantic type of the entity
    "GENE_ID": str,      # The EntrezGene ID of the entity
    "GENE_NAME": str,    # The EntrezGene name of the entity
    "TEXT": str,         # The text in the utterance that maps to the entity
    "START_INDEX": str,  # The first character position (in document) of the text denoting the entity
    "END_INDEX": str,    # The last character position (in document) of the text denoting the entity
    "SCORE": str,        # The confidence score
}

# Found in downloads page (https://lhncbc.nlm.nih.gov/ii/tools/SemRep_SemMedDB_SKR/SemMedDB_download.html)
# Note, Entity file is supposed to have 1,892,228,683 rows, however, a line count using `wc -l` results in 1,887,317,669 new line 
# chars (n-1 rows) being detected on the unmodified file. This is consistent with the data read in after cleaning.
EXPECTED_ROWS = {
    "PREDICATION": 126268045,
    "SENTENCE":    253029872, 
    # "ENTITY":      1892228683, # official amount from linked website.
    "ENTITY":      1887317669 + 1,   # found amount.
}

## File Utils

Over the next few sections, we'll be reading in the raw files to ensure that Dask is capable of properly parsing the minimally-cleaned files. Again, we're not really doing much "cleaning" here, rather verifying that the files we store in our AWS bucket can be properly used later.

In [8]:
import dask.dataframe as dd
import numpy as np
from dask.diagnostics import ProgressBar


def read_csv(filename:str, col_labels:list[str]=None, types:dict=None) -> dd:
    """ Reads in CSV file.

    Args: 
        filename: the path to file to load.
        col_labels: the expected labels for each column. Default None.
        types: column labels and their accompanying types. Default None.
    Returns:
        A Dask Dataframe object with loaded data.
    """
    def print_values(bad_line: list[str]):
        for i,item in enumerate(bad_line):
            print(f"Item {i}: {item}")
            
    opts = {
        "blocksize":        50e6,         # 50 MB chunks
        "dtype":            np.str_,      # treat everything as string for now
        "on_bad_lines":     "warn",
        "encoding":         "iso-8859-1",
        "na_values":        [""],         # representation of empty values
        
        # "engine":       "python",       # uncomment next two lines if debugging / cleaning data
        # "on_bad_lines": print_values,   # warning, will be VERY slow.
    }


    # type checking.
    if col_labels and type(col_labels) != list:
        print(f"Column labels not in proper format. Expected list, got {type(col_labels)}.")
        col_labels = None
    elif col_labels:
        opts["names"] = col_labels
    if types and type(types) != dict:
        print(f"Types are not in proper format. Expected dict, got {type(types)}.")
        types = None
    elif types:
        opts["dtype"] = types

    df = dd.read_csv(filename,**opts)

    return df

### Predications

In [9]:
try:
    predications = read_csv(PREDICATION_FILE, list(PREDICATION_COLUMNS.keys()), PREDICATION_COLUMNS)
except Exception as e:
    print(f"Something seriously messed up happened.\n{e}")

In [4]:
with ProgressBar():
    total_rows = predications.map_partitions(len).compute().sum()

print(f"Lost Rows: {EXPECTED_ROWS['PREDICATION']-total_rows}, Percent lost: {round((1.00 - (total_rows/EXPECTED_ROWS['PREDICATION'])), 3)}")

predications.head()

[########################################] | 100% Completed | 160.24 s
Lost Rows: 0, Percent lost: 0.0


,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
0,10592604,16,16530475,PROCESS_OF,C0003725,Arboviruses,virs,1,C0999630,Lepus capensis,mamm,1
1,10592697,17,16530475,ISA,C0039258,Tahyna virus,virs,1,C0446169,California Group Viruses,virs,1
2,10592728,17,16530475,ISA,C0318627,Eyach virus,virs,1,C0206590,Coltivirus,virs,1
3,10592759,17,16530475,ISA,C0446169,California Group Viruses,virs,1,C0003725,Arboviruses,virs,1
4,10592832,18,16530475,PROCESS_OF,C0012634,Disease,dsyn,0,C0020114,Human,humn,0


### Sentence

In [6]:
try:
    sentences = read_csv(SENTENCE_FILE, list(SENTENCE_COLUMNS.keys()), SENTENCE_COLUMNS)
except Exception as e:
    print(f"Something seriously messed up happened.\n{e}")

In [7]:
with ProgressBar():
    total_rows = sentences.map_partitions(len).compute().sum()

print(f"Lost Rows: {EXPECTED_ROWS['SENTENCE']-total_rows}, Percent lost: {round(1.00 - (total_rows/EXPECTED_ROWS['SENTENCE']), 3)}")

[########################################] | 100% Completed | 417.76 s
Lost Rows: 0, Percent lost: 0.0


### Entities

In [3]:
try:
    entities = read_csv(ENTITY_FILE, list(ENTITY_COLUMNS.keys()), ENTITY_COLUMNS)
except Exception as e:
    print(f"Something seriously messed up happened.\n{e}")

In [4]:
with ProgressBar():
    total_rows = entities.map_partitions(len).compute().sum()

print(f"Lost Rows: {EXPECTED_ROWS['ENTITY']-total_rows}, Percent lost: {round(1.00 - (total_rows/EXPECTED_ROWS['ENTITY']), 3)}")

[########################################] | 100% Completed | 28m 53s
Lost Rows: 0, Percent lost: 0.0


## Uploading to AWS

Here, we use the "Boto3" module to bypass some of the complexity behind uploading to AWS. The secrets to our AWS console session were stored in a `.env` file in the same folder. For more information about the credentials used, please visit the [official boto3 documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html).

We upload pre-zipped versions of the raw csv files as both CSV and Parquet files since the data is relatively large.

In [10]:
from dotenv import dotenv_values
import boto3
import progressbar

env_v = dotenv_values(".env")
class Uploader:

    def __init__(self):
        self.storage_opts= { 
                "aws_access_key_id" : env_v["aws_access_key_id"],
                "aws_secret_access_key": env_v["aws_secret_access_key"],
                "aws_session_token": env_v["aws_session_token"]}
        
        self.s3 = boto3.client('s3', **self.storage_opts)

    def update_pb(self, size):
        self.pg.update(self.pg.currval + size)

    def upload_file(self, file):
        self.pg = progressbar.progressbar.ProgressBar(
            maxval=os.stat(file).st_size)
        self.pg.start()

        self.s3.upload_file(file, 'semdb-data', os.path.basename(file), Callback=self.update_pb)

uploader = Uploader()


storage_opts = {'key': env_v["aws_access_key_id"],
                'secret': env_v["aws_secret_access_key"],
                'token': env_v["aws_session_token"]}

parquet_base_url = 's3://semdb-data/parquet/'

## CSV Uploads

In [7]:
uploader.upload_file(PREDICATION_ZIP)

In [4]:
uploader.upload_file(SENTENCE_ZIP)

In [ ]:
uploader.upload_file(ENTITY_ZIP)

## Parquet Uploads

In [6]:
with ProgressBar():
    s3_url = parquet_base_url+'predication'
    predications.to_parquet(s3_url, storage_options=storage_opts, compression='gzip')

[########################################] | 100% Completed | 48m 23s


In [15]:
with ProgressBar():
    s3_url = parquet_base_url+'sentence'
    sentences.to_parquet(s3_url, storage_options=storage_opts, compression='gzip')

[########################################] | 100% Completed | 3hr 1ms


In [30]:
with ProgressBar():
    s3_url = parquet_base_url+'entity'
    entities.to_parquet(s3_url, storage_options=storage_opts, compression='gzip')

[########################################] | 100% Completed | 5hr 57m
